# HW01
# Student: Pravilov Mikhail

In [106]:
import numpy as np

1. (8 points) Implement the kNN algorithm and calculate Leave-one-out error on both datasets for 1 to 10 neighbors.

Firsly, let's load our dataset.

In [107]:
import pandas as pd
dataset_root="../Datasets/"

In [108]:
# spam.csv loading
spam_path = dataset_root + 'spam.csv'
spam_data = pd.read_csv(spam_path)
spam_data = spam_data.values
spam_X = spam_data[:, 0:-1]
spam_y = spam_data[:, -1]
print("spam datapoints shape: " + str(spam_X.shape))
print("spam labels shape: " + str(spam_y.shape))

spam datapoints shape: (4601, 57)
spam labels shape: (4601,)


In [109]:
# cancer.csv loading
cancer_path = dataset_root + 'cancer.csv'
cancer_data = pd.read_csv(cancer_path)
cancer_data = cancer_data.values
cancer_X = cancer_data[:, 1:]
cancer_y = cancer_data[:, 0]
print("cancer datapoints shape: " + str(cancer_X.shape))
print("cancer labels shape: " + str(cancer_y.shape))

cancer datapoints shape: (569, 30)
cancer labels shape: (569,)


Then let's see what sklearn gives us. This will be a baseline to check that our implementation of kNN is correct.

In [148]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
k_global = 10
kNNs = [KNeighborsClassifier(n_neighbors=i) for i in range(1, k_global + 1)]

Leave one out error is N-fold cross validation, where N is the number of datapoints

In [152]:
def calculate_loo_error_for_classifiers(classifiers, X, y):
    from sklearn.model_selection import cross_val_score, KFold
    return [1 - cross_val_score(skf, X=X, y=y, cv=KFold(n_splits=y.shape[0]), n_jobs=-1).mean() for skf in classifiers]

In [153]:
# cancer dataset
print("Cancer dataset")
print("<num of neighbours> <loo error>")
cancer_sklearn_loo = calculate_loo_error_for_classifiers(kNNs, cancer_X, cancer_y)
for k, loo_error in enumerate(cancer_sklearn_loo):
    print(str(k + 1) + " " + str(loo_error))

Cancer dataset
<num of neighbours> <loo error>
1 0.08435852372583474
2 0.0773286467486819
3 0.07381370826010547
4 0.07381370826010547
5 0.06678383128295251
6 0.07029876977152905
7 0.06854130052724072
8 0.07029876977152905
9 0.06678383128295251
10 0.07029876977152905


In [154]:
# spam dataset
print("Spam dataset")
print("<num of neighbours> <loo error>")
spam_sklearn_loo = calculate_loo_error_for_classifiers(kNNs, spam_X, spam_y)
for k, loo_error in enumerate(spam_sklearn_loo):
    print(str(k + 1) + " " + str(loo_error))

Spam dataset
<num of neighbours> <loo error>
1 0.16952836339926103
2 0.19061073679634866
3 0.18539447946098675
4 0.19300152140838944
5 0.18582916757226686
6 0.20039121930015213
7 0.19582699413171045
8 0.20104325146707236
9 0.20278200391219303
10 0.2021299717452728


Now let's write our own kNN method.

In [155]:
def get_nearest(k, point_index, X):
    return np.argsort(np.array([np.linalg.norm(X[point_index]-x) for x in X]))[1:k + 1]

def generate_nearest(X, k):
    return np.array([get_nearest(k, i, X) for i in range(X.shape[0])])

def get_class(point_index, nearest_matrix, y, k):
    (nearest_classes, counts) = np.unique(y[nearest_matrix[point_index]][:k], return_counts=True)
    majority_index = np.argmax(counts) 
    return nearest_classes[majority_index]    

In [156]:
def leave_one_out(nearest_matrix, y, k):
    return np.mean([get_class(y_index, nearest_matrix, y, k) != y[y_index] for y_index in range(y.shape[0])])

def leave_one_out_for_all_before_k(X, y, k):
    nearest_matrix = generate_nearest(X, k)
    return [leave_one_out(nearest_matrix, y, i) for i in range(1, k + 1)]

In [157]:
# cancer dataset
print("Cancer dataset")
print("<num of neighbours> <loo error>") 
cancer_loo = leave_one_out_for_all_before_k(cancer_X, cancer_y, k_global)
for k, loo_error in enumerate(cancer_loo):
    print(str(k + 1) + " " + str(loo_error))

Cancer dataset
<num of neighbours> <loo error>
1 0.0843585237258348
2 0.0773286467486819
3 0.07381370826010544
4 0.07381370826010544
5 0.06678383128295255
6 0.070298769771529
7 0.06854130052724078
8 0.070298769771529
9 0.06678383128295255
10 0.070298769771529


In [158]:
# spam dataset
print("Spam dataset")
print("<num of neighbours> <loo error>") 
spam_loo = leave_one_out_for_all_before_k(spam_X, spam_y, k_global)
for k, loo_error in enumerate(spam_loo):
    print(str(k + 1) + " " + str(loo_error))

Spam dataset
<num of neighbours> <loo error>
1 0.16865898717670072
2 0.19039339274070854
3 0.18452510323842644
4 0.19213214518582916
5 0.18539447946098675
6 0.199956531188872
7 0.1958269941317105
8 0.2010432514670724
9 0.20256465985655292
10 0.2019126276896327


In [162]:
print("Cancer dataset. my - sklearn")
print(np.array(cancer_sklearn_loo) - np.array(cancer_loo))

Cancer dataset. my - sklearn
[-5.55111512e-17  0.00000000e+00  2.77555756e-17  2.77555756e-17
 -4.16333634e-17  4.16333634e-17 -5.55111512e-17  4.16333634e-17
 -4.16333634e-17  4.16333634e-17]


In [163]:
print("Spam dataset. my - sklearn")
print(np.array(spam_sklearn_loo) - np.array(spam_loo))

Spam dataset. my - sklearn
[ 8.69376223e-04  2.17344056e-04  8.69376223e-04  8.69376223e-04
  4.34688111e-04  4.34688111e-04 -5.55111512e-17 -2.77555756e-17
  2.17344056e-04  2.17344056e-04]


Как видим погрешность маленькая для cancer dataset. Для spam dataset она побольше, но думаю приемлимо и не говорит о том, что есть ошибка в моей имплементации. Связано это наверное с тем, что spam dataset больше и накапливается погрешность при расчете расстояний. Также вероятно в spam dataset датапоинты довольно близки к друг другу, отсюда еще погрешность в выборе k соседей.